In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score
import nltk
from nltk.corpus import stopwords
from langdetect import detect
import re
from translate import Translator
import spacy

In [2]:
df = pd.read_csv('C:/Users/lemes/OneDrive/Рабочий стол/dz/IMDB Dataset.csv')
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [3]:
df.isna().describe()

,review,sentiment
count,50000,50000
unique,1,1
top,False,False
freq,50000,50000


In [4]:
df.groupby('sentiment').count()

,review
sentiment,
negative,25000
positive,25000


In [3]:
lb_encoder = LabelEncoder()
df['sentiment'] = lb_encoder.fit_transform(df['sentiment'])
df.groupby('sentiment').count()

,review
sentiment,
0,25000
1,25000


In [8]:
# витягнемо стоп слова з nltk
nltk.download('stopwords')
words_stop = stopwords.words('english')
words_stop

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lemes\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [7]:
# подивимось, чи всі записи у нас написанні англійскою мовою
df['language'] = df['review'].apply(lambda x: detect(x))
df.groupby('language').count()

,review,sentiment
language,,
en,49999,49999
id,1,1


In [8]:
df.query("language == 'id'")['review']

45315    .....whoops - looks like it's gonna cost you a...
Name: review, dtype: object

In [12]:
def custom_preprocessor(text):
    # видалення всіх символів, окрім букв та пробелів
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    return text

In [10]:
t = ".....whoops - looks like it's gonna cost you a whopping £198.00 to buy a copy (either DVD or Video format)from ITV direct.<br /><br />Ouch.<br /><br />Sorry about this, but IMDB won't let me submi..."
custom_preprocessor(t)

'whoops  looks like its gonna cost you a whopping  to buy a copy either DVD or Video formatfrom ITV directbr br Ouchbr br Sorry about this but IMDB wont let me submi'

In [11]:
model = CountVectorizer(strip_accents='unicode', stop_words=words_stop, preprocessor=custom_preprocessor, min_df=5, dtype=np.int32)
X = model.fit_transform(df['review'])

C:\Users\lemes\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['arent', 'couldnt', 'didnt', 'doesnt', 'dont', 'hadnt', 'hasnt', 'havent', 'isnt', 'mightnt', 'mustnt', 'neednt', 'shant', 'shes', 'shouldnt', 'shouldve', 'thatll', 'wasnt', 'werent', 'wont', 'wouldnt', 'youd', 'youll', 'youre', 'youve'] not in stop_words.
  warnings.warn(


In [12]:
model.get_feature_names_out()

array(['AA', 'AAA', 'AB', ..., 'zoomin', 'zooming', 'zooms'], dtype=object)

In [13]:
train, test = train_test_split(X, train_size=0.5, shuffle=False)

In [4]:
target_train, target_test = train_test_split(df['sentiment'], train_size=0.5, shuffle=False)

In [23]:
def make_clasifier(tree, depth):
    model_clasifier = RandomForestClassifier(n_estimators=tree, max_depth=depth, n_jobs=-1, random_state=13).fit(train, target_train)
    predict_train = model_clasifier.predict(train)
    accuracy_train = accuracy_score(target_train, predict_train)
    precision_train =  precision_score(target_train, predict_train)
    recall_train = recall_score(target_train, predict_train)
    f1_train = f1_score(target_train, predict_train) 
    predict_test = model_clasifier.predict(test)
    accuracy_test = accuracy_score(target_test, predict_test)
    precision_test =  precision_score(target_test, predict_test)
    recall_test = recall_score(target_test, predict_test)
    f1_test = f1_score(target_test, predict_test)
    msg = f'Модель відпрацювала з такими данними:\nнавчальна виборка\naccuracy : {accuracy_train}\nprecision : {precision_train}\nrecall : {recall_train}\nf1 : {f1_train}\n\n\nТестова частина:\naccuracy : {accuracy_test}\nprecision : {precision_test}\nrecall : {recall_test}\nf1 : {f1_test}'
    return {'predict_train':predict_train, 'accuracy_train':accuracy_train, 'precision_train':precision_train, 'recall_train':recall_train, 'f1_train':f1_train, 'predict_test':predict_test, 'accuracy_test':accuracy_test, 'precision_test':precision_test, 'recall_test':recall_test, 'f1_test':f1_test, 'msg':msg, 'model':model_clasifier}
    

In [16]:
max_param = make_clasifier(tree=500,depth=1000)

In [17]:
print(max_param['msg'])

Модель відпрацювала з такими данними:
навчальна виборка
accuracy : 1.0
precision : 1.0
recall : 1.0
f1 : 1.0


Тестова частина:
accuracy : 0.86152
precision : 0.8634322373696872
recall : 0.8595720900526904
f1 : 0.8614978396543447


In [ ]:
acc = []
param = []
for i in range(1,50):
    print(f'Зараз робим {i} ітерацію')
    proba = make_clasifier(tree=500,depth=i)
    acc.append(proba['accuracy_test'])
    param.append([proba['accuracy_test'], proba['precision_test'], proba['recall_test'], proba['f1_test']])

In [ ]:
param[acc.index(max(acc))]

In [ ]:
param

In [ ]:
acc.index(max(acc))
# обиремо глибину наших дерев 43

In [ ]:
# подивимось що видасть нам менше дерев
tree = make_clasifier(tree=500,depth=43)

In [ ]:
acc = []
param = []
for i in range(1,501):
    print(f'Зараз робим {i} ітерацію')
    proba = make_clasifier(tree=i,depth=43)
    acc.append(proba['accuracy_test'])
    param.append([proba['accuracy_test'], proba['precision_test'], proba['recall_test'], proba['f1_test']])

In [ ]:
max(acc)

In [ ]:
param[acc.index(max(acc))]

In [ ]:
acc.index(max(acc))

In [20]:
# Найкращі параметри моделі
base_line = make_clasifier(tree=500,depth=43)

In [21]:
print(base_line['msg'])

Модель відпрацювала з такими данними:
навчальна виборка
accuracy : 0.97924
precision : 0.9649941656942824
recall : 0.9944684944684945
f1 : 0.9795096529669549


Тестова частина:
accuracy : 0.8588
precision : 0.8523973675963648
recall : 0.8685933258821651
f1 : 0.8604191379992092


In [22]:
#Подивимось як він працює, спробуємо написати свій відгук
coments = str(input('Введіть ваш коментар'))
translator = Translator(to_lang="en", from_lang='uk')
translated_comment = translator.translate(coments)
coments = pd.Series({'coments':translated_comment})
my_coment = model.transform(coments)
com_pred = base_line['model'].predict(my_coment)
com_pred = lb_encoder.inverse_transform(com_pred)
com_pred

array(['negative'], dtype=object)

Не дивлячись на те, що телефон коштує 2000 долларів - він задовольняє всім моїм потребам своєю потужністю та зовнішнім виглядом

Жахливий проудкт, я би на такий ніколи не підписався, марно витрачав свої кошти

Вирішив поділитися своїми враженнями від дворічного володінням Cadillac XT5. Рік виготовлення 2017. Повний привід, сучасний атмосферний двигун V6 об’ємом 3,6л., потужність 314 к.с. з системою відключення двох циліндрів, що дозволяє їхати як швидко розгін 7,5с. до 100 км./год.так, і за потреби, економно розхід по трасі може становити 7,5л./100 км.. Був придбаний на аукціоні з незначними пошкодженнями та пробігом 12 474 миль 20 тис.км.в 2021р. За два роки проїхав 12 тис. км. Перед ним у володінні була Ауді А4 2.0 TFSI Quattro, відповідно хотів придбати Audi Q5 до 2017р. випуску, проте потрібного варіанту не знаходилося. Всі Або були надто сильно пошкоджені, або по ціннику йшли далеко поза мій бюджет і ставали нецікавими/невигідними. Вирішив подивитися в бік біль рідкісних авто американського автопрому - Cadillac, Lincoln, Jeep. В результаті став щасливим власником Cadillac XT5 3,6 AWD. Велике, комфортне, непересічне авто, яке, ІМХО, варте уваги і дуже недооцінене у нас. Розміри між Q5 та Q7, великий багажник, хороший кліренс, якісні матеріали оздоблення. Авто абсолютно "не кохає" мізки. Після ремонту замінив рідини та фільтри й після того - виключно те, що потребував регламент. Оскільки це GM вартість фільтрів невелика. Оливу використовував Liqui Moly Molygen New Generation 5W-30, заміна раз на 10 тис. Авто чудово поводить себе як на добрих дорогах так і на недуже. Комфортне в далеких подорожах. Попри те що може швидко їхати, зовсім не провокує, як Ауді. Тут історія про комфорт та зручність. Багато місця в салоні та багажнику. Кліренс достатній для адекватних зїздів з асфальту в якийсь лісок чи за потреби десь проїхати взимку. Багато хто переживатиме за розхід, зважаючи на обєм двигуна - скажу так авто споживає не набагато більше ніж Ауді А4. В залежності від стилю їзди, місто 13-16л., траса 6,5-9 л. Вважаю, що для авто такого обєму/потужності, маси та розмірів це цілком адекватно. Хочу побажати потенційним покупцям даної марки не боятися, це "американський мерседес". З запчастинами зараз проблем немає. А гарний вигляд, непересічність авто точно не залишить вас непоміченими.

Ніколи не звертайтеся до цього салону! ЛОХОТРОН! Розводять на кредит, на машину якої немає! Для того, щоб ви не пішли, утримують документи. Коли розуміють, що ти починаєш розуміти, що це розлучення, починають дуже грубо спілкуватися. Чи не ведіться! Вам розкажуть що машина десь скоро доставлять, тільки подайте на заявку на кредит, тільки ви ніколи не дізнаєтесь на яку суму вас подали.

На жаль Спортлайф клуб для тих, хто не працює та особливо це стосуватися дітей. Навіщо взагалі продавати дитячий дитячий абонемент, якщо зверху за все потрібно доплачувати.
В мене дитина 9 років відходила зі мною рік, коли ми працювали вдома, була можливість його водити, підлаштовуючи графік. При вимушеній роботі в офісі, всі заняття в робочий час. Завезти дитину на групове заняття по абонементу може або ти з наявністю або той дорослий, який має абонемент, або супровід за 80 грн(і це за 5 хв щоб провести латину через роздягальні до тренера), або платіть за платне групове(і тоді вийде тренер). Тобто альтернативи, щоб вийшов або зустрів тренер, який проводить заняття по абонементу - немає. Платіть зверху гроші. Висновок, дитина не хоче ходити. Домучаєм абонемент і більше не куплятимо.
Вимагаєте договір, коли оплачуєте абонемент.
Бо дитині нічого не можно, лише знаходитись в дитячій кімнаті та з дорослим, але ви тоді не тренуватись.

Спорт лайф-то найгірша пропозиція в сучасному світі, мало того, що сервіс гірше не придумаєш ( тренери так собі, розклад групових лише для пенсіонерів та школярів) тай ще клуб на Дарниці закрили без попередження та права на заморозку у разі як що не хочу їздити в інший клуб, і це при такій вартості абонементів!!! Це все одно що в тебе вкрали півтора місяця від вартості абонементу в розмірі щонайменше 19 тис гривень.

Перевірив на своїх коментарях - все працює (насправді я трошки в захваті)

Спроюуємо іншу бібліотеки scikit-learn

In [23]:
model2 = TfidfVectorizer(strip_accents='unicode', stop_words=words_stop, preprocessor=custom_preprocessor, min_df=5, dtype=np.int32, norm='l1')
X2 = model2.fit_transform(df['review'])

C:\Users\lemes\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:2065: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. <class 'numpy.int32'> 'dtype' will be converted to np.float64.
  warnings.warn(
C:\Users\lemes\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['arent', 'couldnt', 'didnt', 'doesnt', 'dont', 'hadnt', 'hasnt', 'havent', 'isnt', 'mightnt', 'mustnt', 'neednt', 'shant', 'shes', 'shouldnt', 'shouldve', 'thatll', 'wasnt', 'werent', 'wont', 'wouldnt', 'youd', 'youll', 'youre', 'youve'] not in stop_words.
  warnings.warn(


In [24]:
train, test = train_test_split(X2, train_size=0.5, shuffle=False)

In [25]:
max_param2 = make_clasifier(tree=500,depth=1000)

In [ ]:
print(max_param2['msg']) # use l2

In [26]:
print(max_param2['msg']) # use l1

Модель відпрацювала з такими данними:
навчальна виборка
accuracy : 1.0
precision : 1.0
recall : 1.0
f1 : 1.0


Тестова частина:
accuracy : 0.8586
precision : 0.8605341246290801
recall : 0.8566182340731279
f1 : 0.8585717143428685


In [ ]:
#Залишимо l1 клфсифікацію
max_param2 = make_clasifier(tree=500,depth=50)
#подивимось невелику глибину

In [ ]:
print(max_param2['msg'])

In [ ]:
acc = []
param = []
for i in range(1,50):
    print(f'Зараз робим {i} ітерацію')
    proba = make_clasifier(tree=500,depth=i)
    acc.append(proba['accuracy_test'])
    param.append([proba['accuracy_test'], proba['precision_test'], proba['recall_test'], proba['f1_test']])

In [ ]:
# будемо обирати глибину 50

In [ ]:
proba = make_clasifier(tree=700,depth=50)

In [ ]:
print(proba['msg'])

In [27]:
base_line2 = make_clasifier(tree=700,depth=50)

In [28]:
#Подивимось як він працює, спробуємо написати свій відгук
coments = str(input('Введіть ваш коментар'))
translator = Translator(to_lang="en", from_lang='uk')
translated_comment = translator.translate(coments)
coments = pd.Series({'coments':translated_comment})
my_coment = model2.transform(coments)
com_pred = base_line2['model'].predict(my_coment)
com_pred = lb_encoder.inverse_transform(com_pred)
com_pred

array(['positive'], dtype=object)

Перейдемо до spacy

In [5]:
nlp = spacy.load('en_core_web_lg')
#doc = nlp(df['review'])

In [5]:
def vectorize_spacy(text):
    doc = nlp(text)
    vector = []
    for sentence in doc.sents:
        for token in sentence:
            if not token.is_punct or token.text.lower() not in words_stop:
                vector.extend(token.vector)            
    vector_arry = np.array(vector, dtype='float32')
    return vector_arry

In [6]:
train1, test = train_test_split(df, train_size=0.5, shuffle=False)

In [7]:
train, test = train_test_split(train1, train_size=0.5, shuffle=False)

In [11]:
target_train = train['sentiment']
train.drop('sentiment', axis=1, inplace=True)
target_test = test['sentiment']
test.drop('sentiment', axis=1, inplace=True)

In [22]:
train_sc = train['review'].apply(vectorize_spacy)

In [24]:
test_sc = test['review'].apply(vectorize_spacy)

In [60]:
train = train_sc.apply(np.mean)
test = test_sc.apply(np.mean)

In [67]:
train = train.values.reshape(-1, 1)

In [69]:
test = test.values.reshape(-1,1)

In [104]:
def make_clasifier_sc():
    model_clasifier = KNeighborsClassifier(n_neighbors=4).fit(train, target_train)
    predict_train = model_clasifier.predict(train)
    accuracy_train = accuracy_score(target_train, predict_train)
    precision_train =  precision_score(target_train, predict_train)
    recall_train = recall_score(target_train, predict_train)
    f1_train = f1_score(target_train, predict_train) 
    predict_test = model_clasifier.predict(test)
    accuracy_test = accuracy_score(target_test, predict_test)
    precision_test =  precision_score(target_test, predict_test)
    recall_test = recall_score(target_test, predict_test)
    f1_test = f1_score(target_test, predict_test)
    msg = f'Модель відпрацювала з такими данними:\nнавчальна виборка\naccuracy : {accuracy_train}\nprecision : {precision_train}\nrecall : {recall_train}\nf1 : {f1_train}\n\n\nТестова частина:\naccuracy : {accuracy_test}\nprecision : {precision_test}\nrecall : {recall_test}\nf1 : {f1_test}'
    return {'predict_train':predict_train, 'accuracy_train':accuracy_train, 'precision_train':precision_train, 'recall_train':recall_train, 'f1_train':f1_train, 'predict_test':predict_test, 'accuracy_test':accuracy_test, 'precision_test':precision_test, 'recall_test':recall_test, 'f1_test':f1_test, 'msg':msg, 'model':model_clasifier}

In [105]:
proba_sc = make_clasifier_sc()

In [106]:
print(proba_sc['msg'])

Модель відпрацювала з такими данними:
навчальна виборка
accuracy : 0.70152
precision : 0.8101805416248746
recall : 0.5207091055600322
f1 : 0.6339644854311782


Тестова частина:
accuracy : 0.532
precision : 0.5524931094963669
recall : 0.3517307385547934
f1 : 0.4298245614035088


Можна стверджумати, що данний підхід не працює, але, зробимо лемітізацію та подивимось

In [6]:
def lema_sc(text):
    doc = nlp(text)
    vector = []
    for sentence in doc.sents:
        for token in sentence:
            if not token.is_punct or token.text.lower() not in words_stop:
                vector.append(token.lemma_)            
    return ' '.join(vector).replace('<br />', '')

In [9]:
sc_df = df['review'].apply(lema_sc)

In [10]:
sc_df

0        one of the other reviewer have mention that af...
1        a wonderful little production . < br />the fil...
2        I think this be a wonderful way to spend time ...
3        basically there be a family where a little boy...
4        petter Mattei 's " love in the Time of money "...
                               ...                        
49995    I think this movie do a down right good job . ...
49996    bad plot , bad dialogue , bad act , idiotic di...
49997    I be a Catholic teach in parochial elementary ...
49998    I be go to have to disagree with the previous ...
49999    no one expect the Star Trek movie to be high a...
Name: review, Length: 50000, dtype: object

In [13]:
model3 = TfidfVectorizer(strip_accents='unicode', stop_words=words_stop, preprocessor=custom_preprocessor, min_df=5, dtype=np.int32, norm='l1')
X3 = model3.fit_transform(sc_df)

C:\Users\lemes\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:2065: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. <class 'numpy.int32'> 'dtype' will be converted to np.float64.
  warnings.warn(
C:\Users\lemes\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['arent', 'couldnt', 'didnt', 'doesnt', 'dont', 'hadnt', 'hasnt', 'havent', 'isnt', 'mightnt', 'mustnt', 'neednt', 'shant', 'shes', 'shouldnt', 'shouldve', 'thatll', 'wasnt', 'werent', 'wont', 'wouldnt', 'youd', 'youll', 'youre', 'youve'] not in stop_words.
  warnings.warn(


In [18]:
type(X3)

scipy.sparse._csr.csr_matrix

In [19]:
train, test = train_test_split(X3, test_size=0.5, shuffle=False)

In [24]:
max_param3 = make_clasifier(tree=500,depth=50)

In [25]:
print(max_param3['msg'])

Модель відпрацювала з такими данними:
навчальна виборка
accuracy : 0.995
precision : 0.9902342199285431
recall : 0.9998396665063332
f1 : 0.9950137620168334


Тестова частина:
accuracy : 0.85608
precision : 0.8532763532763533
recall : 0.8607695992335941
f1 : 0.8570065972498211


Висновок - для нашої задачі достатньо звичайного Bag of Words